<h1 align="center">随机数的应用</h1>

# 理论

[主要来自统计模拟及其R实现(武汉大学出版社, 2010)第3章]

令 $g(X)$是一个函数, 假设我们要计算$\theta$:

$$
\theta = \int_0^1 g(x) dx
$$

- 若$U$服从$(0,1)$上的均匀分布, 那么就有$\theta = E[g(U)]$.
- 若$U_1, U_2, \cdots, U_k$是$(0,1)$上的均匀分布, 则有$g(U_1), g(U_2), \cdots, g(U_k)$是均值为$\theta$的独立同分布的变量.
- 由强大数定律, 以概率1有

$$
\sum_{i=1}^k \frac{g(U_i)}{k} \to E[g(U)] = \theta , \text{while} \; \; k \to \infty
$$

- 故能通过该方法产生大量随机数, 且取$g(U_i)$的平均值作为近似值
- 该方法称为MonteCarlo方法.
- **该部分的关键为如何将其转化为标准的形式:$\theta = \int_0^1 g(x)dx$**

# 习题

该部分习题主要选自统计模拟及其R实现(武汉大学出版社, 2010)第3章

## 3.8 

用模拟的方法近似计算下列积分,并和已知的精确答案进行比较.

- (1) $\int_{0}^{1}\left(1-x^{4}\right)^{3} \mathrm{~d} x$
- (2) $\int_{0}^{\infty} x\left(1+x^{3}\right)^{-4} \mathrm{~d} x$
- (3) $\int_{-\infty}^{\infty} \exp \left\{-x^{2}\right\} \mathrm{d} x$
- (4) $\int_{0}^{\infty} \int_{0}^{x} 2 e^{-(2 x+y)} d y d x$
[ 提示 : 令 $I_{y}(x)=\left\{\begin{array}{l}1, \text { 若 } y<x \\ 0, \text { 否则 }\end{array}\right.$ ]

### 答案

#### (1)

In [1]:
# 导入需要用的包
import numpy as np
from numpy import random
from math import *
from scipy import integrate

In [6]:
# 首先使用integrate计算得出精确解
gx = lambda x: (1-x**4)**3
integrate.quad(gx,0,1) 

(0.6564102564102564, 7.287617802667694e-15)

使用MC方法得到积分: 
- 首先产生均匀分布随机数
- 代入`g()`函数, 求出其平均值

In [8]:
def eg3_8_1(size, low, high, func):
    """
    - Input:
        - size: The number of random numbers generated
        - lower: Integral lower bound
        - higher: Integral higher bound
        - func: Function of g(x) mentioned above
    - Return:
        - mu: Estimate
    """
    out = random.uniform(size = size)
    mu = np.mean(
        (high-low) * gx(low + (high-low)*out)
    )

    return mu

random.seed(0)
eg3_8_1(10000, 0, 1, gx)

0.659848955067479

#### (2)

- 此处逻辑与上一题相似, 首先求得精确解, 再通过模拟得到模拟值, 并于精确解进行比较.
- 此处涉及了无穷积分, 所以需要对其进行变换才能得到相应的答案.

In [10]:
gx = lambda x: x*(1+x**3)**(-4)
integrate.quad(gx, 0, np.inf)

(0.20899745760723498, 3.1354534066841486e-09)

首先需要对替换x:
- 令$y = \frac{1}{x+1}$
- $dy = -\frac{dx}{(x+1)^2} = -y^2dx$
- $\theta = \int_0^1\frac{g(y(x)^{-1})}{y^2}dy$

**通过该题目进行总结: 使用$y=\frac{1}{x+1}$对$[0,+\infty]$进行放缩**

In [60]:
dy_opposite = lambda y: y**2
y_inverse = lambda y: 1/y-1

def eg3_8_2(size, gx, dy_opposite, y_inverse):
    y = random.uniform(size = size)
    x = gx(y_inverse(y))/dy_opposite(y)

    return np.mean(x)

eg3_8_2(500000, gx, dy_opposite, y_inverse)

0.2091994137082532

#### (3)

此处使用偶函数的性质, 其余内容和(2)相同

In [62]:
# 精确解

gx = lambda x: np.exp(-x**2)

integrate.quad(gx, -np.inf, +np.inf)

(1.7724538509055159, 1.4202636781830878e-08)

In [61]:

def eg3_8_2(size, gx, dy_opposite, y_inverse):
    y = random.uniform(size = size)
    x = gx(y_inverse(y))/dy_opposite(y)

    return np.mean(x)

# 最后需要在结果中*2
2 * eg3_8_2(500000, gx, dy_opposite, y_inverse)

1.7699457915746983

#### (4)

----

**计算多重积分的方法:**

计算
$$
\theta = \int_0^1 \int_0^1 \cdots \int_0^1 = g(x_1, x_2, \cdots, x_n)dx_1dx_2\cdots dx_n
$$
可以使用MonteCarlo方法进行估计:
$$
\theta = E[g(U_1, U_2, \cdots, U_n)]
$$
其中$U_1, U_2, \cdots,U_n$是相互独立的(0,1)上的均匀分布.

----

在该小问中:
- 此时的主要内容为二重积分
- 关键点在于如何将x化为(0, 1)上的值

所以, 根据提示可以得到:

$$
\int_0^\infty \int_0^x 2e^{-(2x+y)}dydx = \int_0^\infty \int_0^\infty 2e^{-(2x+y)}\cdot I_y(x)dydx
$$

此时可以通过(2), (3)的变换思路求得解答

In [123]:
I = lambda x, y: 1 if y < x else 0
fx = lambda y, x: 2*np.exp(-2*x-y)
gx = lambda x, y: fx(x, y)*I(x, y)

integrate.dblquad(fx, 0, np.inf, 0, lambda x:x)

(0.33333333333333315, 1.5988016020128278e-10)

In [130]:
I = lambda x, y: np.where(x<y, 0, 1)
fx = lambda x, y: 2*np.exp(-2*x-y)
gx = lambda x, y: fx(x, y)*I(x, y)

def eg3_8_4(size, gx):
    x, y = random.uniform(size = size), random.uniform(size = size)
    out = gx(1/x-1,1/y-1)/(x**2 * y**2)
    return np.mean(out)

eg3_8_4(500000, gx)

0.33261384054861776

## 3.9

用随机模拟的方法计算 $\operatorname{Cov}\left(U, \mathrm{e}^{U}\right)$, 其中 $U$ 是 $(0,1)$ 上的均匀随机变量,并和你的精确答案进行比较.

### 答案

In [136]:
# 模拟解
U = random.uniform(size=100000)
Cov = np.mean(U * np.exp(U)) - np.mean(U) * np.mean(np.exp(U))
Cov

0.14035181020332033

In [139]:
# 精确解

integrate.quad(lambda x: x*exp(x)-0.5*exp(x), 0, 1)

(0.1408590857704774, 4.84907016207406e-15)